In [29]:
import numpy as np
import pandas as pd
pd. set_option('display.max_columns', None)
from sklearn.preprocessing import LabelEncoder
import datetime as dt
import xgboost
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold


In [5]:
train_df= pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [6]:
train_df.head()

,tracking_id,datetime,wind_speed(m/s),atmospheric_temperature(°C),shaft_temperature(°C),blades_angle(°),gearbox_temperature(°C),engine_temperature(°C),motor_torque(N-m),generator_temperature(°C),atmospheric_pressure(Pascal),area_temperature(°C),windmill_body_temperature(°C),wind_direction(°),resistance(ohm),rotor_torque(N-m),turbine_status,cloud_level,blade_length(m),blade_breadth(m),windmill_height(m),windmill_generated_power(kW/h)
0,WM_33725,2019-08-04 14:33:20,94.820023,-99.000000,41.723019,-0.903423,82.410573,42.523015,2563.124522,76.665560,103402.961872,26.897875,NaN,239.836388,2730.310605,42.084666,BA,Medium,2.217542,0.314065,24.281689,6.766521
1,WM_698,2018-11-05 10:13:20,241.832734,27.764785,-99.000000,-99.000000,44.104919,46.258870,2372.384119,78.129803,17030.904078,39.801469,NaN,337.944723,1780.207200,107.888643,A2,Medium,4.210346,0.448494,27.262139,5.966275
2,WM_39146,2019-09-14 14:03:20,95.484724,NaN,41.855473,12.652763,42.322098,42.878552,1657.169646,67.654469,16125.927107,36.116065,45.033197,227.850294,1666.049900,-42.931459,ABC,Medium,2.719475,0.302321,27.366127,2.874342
3,WM_6757,2018-12-25 15:33:20,238.819424,-99.000000,45.443914,15.115323,44.759643,47.282101,2888.134079,95.389974,18689.732336,46.020045,44.827154,492.081520,1964.502895,42.744596,ABC,NaN,4.857385,0.367140,24.287767,14.851089
4,WM_21521,2019-05-04 03:13:20,10.722890,NaN,41.981183,1.715696,-17.616459,43.469852,781.695419,37.423065,114468.169007,34.572941,-99.000000,259.274601,1177.516152,13.387289,AAA,Medium,NaN,0.453374,27.971650,3.519074


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28200 entries, 0 to 28199
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   tracking_id                     28200 non-null  object 
 1   datetime                        28200 non-null  object 
 2   wind_speed(m/s)                 27927 non-null  float64
 3   atmospheric_temperature(°C)     24750 non-null  float64
 4   shaft_temperature(°C)           28198 non-null  float64
 5   blades_angle(°)                 27984 non-null  float64
 6   gearbox_temperature(°C)         28199 non-null  float64
 7   engine_temperature(°C)          28188 non-null  float64
 8   motor_torque(N-m)               28176 non-null  float64
 9   generator_temperature(°C)       28188 non-null  float64
 10  atmospheric_pressure(Pascal)    25493 non-null  float64
 11  area_temperature(°C)            28200 non-null  float64
 12  windmill_body_temperature(°C)   

In [9]:
train_df['turbine_status'].value_counts()

BB     1946
AAA    1939
BCB    1933
B2     1931
A      1930
D      1922
B      1882
AB     1868
ABC    1867
A2     1855
BA     1854
AC     1850
BD     1843
BBB    1821
Name: turbine_status, dtype: int64

In [11]:
train_df['cloud_level'].value_counts()

Low              13921
Medium           13704
Extremely Low      299
Name: cloud_level, dtype: int64

In [12]:
label_encoder1 = LabelEncoder()
label_encoder2 = LabelEncoder()

In [14]:
train_df['turbine_status'] = label_encoder1.fit_transform(train_df['turbine_status'].astype(str))
test_df['turbine_status'] = label_encoder1.fit_transform(test_df['turbine_status'].astype(str))

train_df['cloud_level'] = label_encoder2.fit_transform(train_df['cloud_level'].astype(str))
test_df['cloud_level'] = label_encoder2.fit_transform(test_df['cloud_level'].astype(str))

In [15]:
train_date_time = pd.to_datetime(train_df['datetime'])
train_df.insert (2, "datetime1", train_date_time)
train_df['datetime1']=train_df['datetime1'].map(dt.datetime.toordinal)

test_date_time= pd.to_datetime(test_df['datetime'])
test_df.insert (2, "datetime1", test_date_time)
test_df['datetime1']=test_df['datetime1'].map(dt.datetime.toordinal)

In [16]:
train_df

,tracking_id,datetime,datetime1,wind_speed(m/s),atmospheric_temperature(°C),shaft_temperature(°C),blades_angle(°),gearbox_temperature(°C),engine_temperature(°C),motor_torque(N-m),generator_temperature(°C),atmospheric_pressure(Pascal),area_temperature(°C),windmill_body_temperature(°C),wind_direction(°),resistance(ohm),rotor_torque(N-m),turbine_status,cloud_level,blade_length(m),blade_breadth(m),windmill_height(m),windmill_generated_power(kW/h)
0,WM_33725,2019-08-04 14:33:20,737275,94.820023,-99.000000,41.723019,-0.903423,82.410573,42.523015,2563.124522,76.665560,103402.961872,26.897875,NaN,239.836388,2730.310605,42.084666,8,2,2.217542,0.314065,24.281689,6.766521
1,WM_698,2018-11-05 10:13:20,737003,241.832734,27.764785,-99.000000,-99.000000,44.104919,46.258870,2372.384119,78.129803,17030.904078,39.801469,NaN,337.944723,1780.207200,107.888643,1,2,4.210346,0.448494,27.262139,5.966275
2,WM_39146,2019-09-14 14:03:20,737316,95.484724,NaN,41.855473,12.652763,42.322098,42.878552,1657.169646,67.654469,16125.927107,36.116065,45.033197,227.850294,1666.049900,-42.931459,4,2,2.719475,0.302321,27.366127,2.874342
3,WM_6757,2018-12-25 15:33:20,737053,238.819424,-99.000000,45.443914,15.115323,44.759643,47.282101,2888.134079,95.389974,18689.732336,46.020045,44.827154,492.081520,1964.502895,42.744596,4,3,4.857385,0.367140,24.287767,14.851089
4,WM_21521,2019-05-04 03:13:20,737183,10.722890,NaN,41.981183,1.715696,-17.616459,43.469852,781.695419,37.423065,114468.169007,34.572941,-99.000000,259.274601,1177.516152,13.387289,2,2,NaN,0.453374,27.971650,3.519074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28195,WM_7814,2019-01-02 02:43:20,737061,94.765699,23.576793,45.399352,5.377222,-1.085171,48.528248,2791.600990,90.898875,19428.725079,45.429230,44.242269,536.153413,1980.861921,45.909054,9,2,2.774335,0.418299,24.590801,9.587934
28196,WM_32512,2019-07-26 12:53:20,737266,94.196738,24.034329,42.068979,-99.000000,44.285153,43.487939,2207.882276,72.244645,16596.485400,25.142681,43.616412,354.239825,1712.840457,36.974913,9,1,-3.250989,0.461531,26.051604,4.522195
28197,WM_5193,2018-12-12 02:13:20,737040,94.160463,28.674296,45.004213,9.550358,49.377706,44.042632,2801.657374,94.814637,19083.881449,45.129442,43.576510,534.209913,1951.728713,88.319152,13,2,NaN,0.380264,28.533850,11.096599
28198,WM_12173,2019-02-03 19:13:20,737093,95.430377,26.560254,48.032624,3.051389,81.443896,44.821365,2760.647280,90.144418,18360.785707,45.603927,44.973415,568.504898,1968.917692,47.562627,11,1,3.001855,0.346447,47.747269,9.373239


In [23]:
train_df.fillna(0,inplace=True)

test_df.fillna(0,inplace=True)

In [24]:
train_df.replace([-99],[0],inplace=True)

test_df.replace([-99],[0],inplace=True)

In [25]:
x_train=train_df.iloc[:,2:-1]
y_train=train_df.iloc[:,-1:]

In [26]:
x_train.head(5)

,datetime1,wind_speed(m/s),atmospheric_temperature(°C),shaft_temperature(°C),blades_angle(°),gearbox_temperature(°C),engine_temperature(°C),motor_torque(N-m),generator_temperature(°C),atmospheric_pressure(Pascal),area_temperature(°C),windmill_body_temperature(°C),wind_direction(°),resistance(ohm),rotor_torque(N-m),turbine_status,cloud_level,blade_length(m),blade_breadth(m),windmill_height(m)
0,737275,94.820023,0.000000,41.723019,-0.903423,82.410573,42.523015,2563.124522,76.665560,103402.961872,26.897875,0.000000,239.836388,2730.310605,42.084666,8,2,2.217542,0.314065,24.281689
1,737003,241.832734,27.764785,0.000000,0.000000,44.104919,46.258870,2372.384119,78.129803,17030.904078,39.801469,0.000000,337.944723,1780.207200,107.888643,1,2,4.210346,0.448494,27.262139
2,737316,95.484724,0.000000,41.855473,12.652763,42.322098,42.878552,1657.169646,67.654469,16125.927107,36.116065,45.033197,227.850294,1666.049900,-42.931459,4,2,2.719475,0.302321,27.366127
3,737053,238.819424,0.000000,45.443914,15.115323,44.759643,47.282101,2888.134079,95.389974,18689.732336,46.020045,44.827154,492.081520,1964.502895,42.744596,4,3,4.857385,0.367140,24.287767
4,737183,10.722890,0.000000,41.981183,1.715696,-17.616459,43.469852,781.695419,37.423065,114468.169007,34.572941,0.000000,259.274601,1177.516152,13.387289,2,2,0.000000,0.453374,27.971650


In [37]:
regressor=xgboost.XGBRegressor()

# Hyper Parameter Optimization

In [31]:
n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3,4]
base_score=[0.25,0.5,0.75,1]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }

In [33]:
%%time

kfold = KFold(n_splits=5, shuffle=True, random_state=10)
grid_search = RandomizedSearchCV(regressor, hyperparameter_grid, verbose = 5,n_iter = 10, cv=kfold,n_jobs=-1)
grid_result = grid_search.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   35.7s
[Parallel(n_jobs=-1)]: Done  46 out of  50 | elapsed: 13.0min remaining:  1.1min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 13.2min finished


Wall time: 13min 17s


In [34]:
grid_result.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=5,
             min_child_weight=3, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [35]:
grid_result.best_params_

{'n_estimators': 100,
 'min_child_weight': 3,
 'max_depth': 5,
 'learning_rate': 0.2,
 'booster': 'gbtree',
 'base_score': 0.5}

In [41]:
regressor=xgboost.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=3,
             min_child_weight=3, monotone_constraints='()',
             n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [42]:
regressor.fit(x_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=3,
             min_child_weight=3, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=12, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [43]:
test_df['windmill_generated_power(kW/h)']=regressor.predict(test_df.drop(['tracking_id','datetime'],axis=1))# predictions
test_df

,tracking_id,datetime,datetime1,wind_speed(m/s),atmospheric_temperature(°C),shaft_temperature(°C),blades_angle(°),gearbox_temperature(°C),engine_temperature(°C),motor_torque(N-m),generator_temperature(°C),atmospheric_pressure(Pascal),area_temperature(°C),windmill_body_temperature(°C),wind_direction(°),resistance(ohm),rotor_torque(N-m),turbine_status,cloud_level,blade_length(m),blade_breadth(m),windmill_height(m),windmill_generated_power(kW/h)
0,WM_19817,2019-04-17 08:53:20,737166,94.324266,17.641186,89.714193,51.146788,40.460560,39.594734,1073.202715,66.830037,16681.044117,24.004812,43.756693,445.976992,1664.222023,21.912243,8,2,3.185837,0.403965,25.572431,2.425361
1,WM_18723,2019-03-30 07:43:20,737148,10.088870,13.978119,43.272846,46.516394,40.027788,41.176860,517.436430,37.284163,0.000000,29.431813,42.728174,499.595287,1165.111992,-35.050093,0,2,3.016603,0.444755,24.371823,2.516182
2,WM_34552,2019-08-10 11:33:20,737281,347.152090,31.423035,41.076640,26.931602,43.109122,43.439556,1480.716492,70.010762,214812.836175,29.924235,43.256122,245.432231,1667.720491,27.195302,7,2,2.611941,0.387368,27.654677,3.044569
3,WM_28570,2019-06-26 03:53:20,737236,24.471997,0.000000,14.375078,66.513953,13.741253,15.577472,887.979475,41.445258,0.000000,23.886434,13.501595,0.000000,1329.744740,15.245757,10,1,2.866805,0.450478,24.189426,7.193809
4,WM_36934,2019-08-27 16:43:20,737298,96.997026,33.281836,41.405192,1.843112,121.572907,43.934587,2053.916354,68.007787,16833.546517,35.906889,0.000000,442.425744,691.408996,34.257024,0,1,3.549672,0.368355,4.885440,3.693276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12081,WM_13376,2019-02-12 11:33:20,737102,-15.039400,18.517204,43.545409,-1.094038,44.550547,44.486619,2424.370244,75.850209,17239.345506,28.424722,42.128526,509.795489,1798.141681,-14.365696,14,2,3.286975,0.407771,20.888674,6.926880
12082,WM_1630,2018-11-12 17:33:20,737010,97.594116,0.000000,44.747916,1.750553,43.308176,42.848779,2085.627252,73.091866,17131.448669,50.615296,47.353528,345.705839,1680.245230,-7.062661,3,1,0.000000,0.367823,24.793606,3.245784
12083,WM_24703,2019-05-27 11:53:20,737206,16.186805,8.109976,43.252012,-0.691090,214.904165,41.177782,788.687177,39.514258,118707.643665,20.983939,41.768478,205.569627,2343.165194,12.331790,8,1,3.728375,0.459887,5.653451,5.250509
12084,WM_22893,2019-05-13 21:53:20,737192,93.254033,0.000000,43.841789,-1.171974,41.976913,42.039796,2079.855634,71.482324,17155.269313,29.114305,82.750271,260.578999,1732.883672,0.000000,0,2,2.247676,0.394680,24.591364,5.221199


In [44]:
submit_df=test_df[['tracking_id','datetime','windmill_generated_power(kW/h)']]
submit_df

,tracking_id,datetime,windmill_generated_power(kW/h)
0,WM_19817,2019-04-17 08:53:20,2.425361
1,WM_18723,2019-03-30 07:43:20,2.516182
2,WM_34552,2019-08-10 11:33:20,3.044569
3,WM_28570,2019-06-26 03:53:20,7.193809
4,WM_36934,2019-08-27 16:43:20,3.693276
...,...,...,...
12081,WM_13376,2019-02-12 11:33:20,6.926880
12082,WM_1630,2018-11-12 17:33:20,3.245784
12083,WM_24703,2019-05-27 11:53:20,5.250509
12084,WM_22893,2019-05-13 21:53:20,5.221199


In [45]:
submit_df.to_csv('test_saquib_1.csv',index=False)